# Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
import load_script

In [3]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [4]:
train_dataset,target=load_script.load_dataset(train=True)

In [5]:
test_dataset,test_target=load_script.load_dataset(train=False)

In [14]:
train_dataset[0]

Variable containing:
  41.8000   44.8000   47.1000  ...    69.8000   72.6000   76.1000
 -10.3000   -5.9000   -3.3000  ...    12.6000   24.0000   26.5000
  38.1000   25.2000   46.0000  ...    45.1000   74.1000   64.8000
             ...                ⋱                ...             
   7.9000   11.2000   14.3000  ...    32.7000   43.4000   45.5000
  19.2000   33.6000   33.8000  ...    46.7000   53.7000   43.4000
  -0.4000   12.7000   12.0000  ...    30.7000   40.6000   33.1000
[torch.FloatTensor of size 28x50]

In [7]:
target

Variable containing:
    1     0
    0     1
    1     0
    0     1
    1     0
    0     1
    1     0
    0     1
    1     0
    1     0
    0     1
    0     1
    1     0
    1     0
    0     1
    0     1
    0     1
    1     0
    1     0
    0     1
    1     0
    1     0
    0     1
    1     0
    0     1
    1     0
    1     0
    1     0
    1     0
    1     0
    1     0
    1     0
    0     1
    0     1
    0     1
    0     1
    0     1
    0     1
    1     0
    0     1
    1     0
    0     1
    1     0
    1     0
    0     1
    1     0
    0     1
    0     1
    1     0
    0     1
    0     1
    1     0
    1     0
    0     1
    1     0
    1     0
    0     1
    1     0
    0     1
    1     0
    0     1
    1     0
    1     0
    0     1
    1     0
    1     0
    1     0
    1     0
    1     0
    0     1
    1     0
    0     1
    1     0
    1     0
    1     0
    1     0
    1     0
    0     1
    0     1
    0     1
    1     0
    0  

# Defining the network

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 28 * 50
#         self.conv1=nn.Conv2d(1,64,(5,3),padding=0)
#         # 12 * 24
#         self.conv2=nn.Conv2d(64,128,3,padding=0)
        # 5 * 10
#         self.full1=nn.Linear(64 * 12 * 23,100)

        self.conv1=nn.Conv1d(28,28,5)
        self.conv2=nn.Conv1d(28,28,5)
        self.full1=nn.Linear(28*42,50)
        self.full2=nn.Linear(50,2)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=F.max_pool1d(F.relu(self.conv2(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x))
        x=F.sigmoid(self.full2(x))
        
        return x

In [25]:
net=Net()
net

Net(
  (conv1): Conv1d(28, 28, kernel_size=(5,), stride=(1,))
  (conv2): Conv1d(28, 28, kernel_size=(5,), stride=(1,))
  (full1): Linear(in_features=1176, out_features=50, bias=True)
  (full2): Linear(in_features=50, out_features=2, bias=True)
)

# Training

In [26]:
type(train_dataset.data)

torch.FloatTensor

In [27]:
trains=train_dataset[:3]

In [28]:
net=Net()

In [29]:
trains=trains.view([3,1,28,50])

In [33]:
net=Net()

optimizer = optim.SGD(net.parameters(), lr=0.0001)
criterion=nn.MSELoss()

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(target.shape[0])
    for i, pair in enumerate(zip(train_dataset,target)):
        
        train_element,target_element=pair
        train_element=train_element.view(1,28,-1)
        
        optimizer.zero_grad()
        
        out=net(train_element)
        output_target[i]=int(out[1]>out[0])
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(test_target.shape[0])
    for i,pair in enumerate(zip(test_dataset,test_target)):
        test_element,test_target_element=pair
        test_element=test_element.view(1,28,-1)
        out=net(test_element)
#         print(out)
        output_test[i]=int(out[1]>out[0])
        
    
        
        
    error_train=np.sum(list(output_target==target[:,1].data))/target.shape[0]
    error_test=np.sum(list(output_test==test_target[:,1].data))/test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")

Epoch: 0 Loss: 87.79267921298742 Correct: 46.83% Correct test: 52.0%
Epoch: 10 Loss: 69.33674959465861 Correct: 65.50% Correct test: 51.0%
Epoch: 20 Loss: 61.38990478403866 Correct: 73.73% Correct test: 55.0%
Epoch: 30 Loss: 53.43297802237794 Correct: 79.11% Correct test: 57.0%
Epoch: 40 Loss: 45.824691348243505 Correct: 85.12% Correct test: 59.0%
Epoch: 50 Loss: 38.79794893489452 Correct: 88.29% Correct test: 61.0%
Epoch: 60 Loss: 32.97417310725723 Correct: 91.45% Correct test: 64.0%
Epoch: 70 Loss: 28.115691689792584 Correct: 92.08% Correct test: 65.0%
Epoch: 80 Loss: 23.89216627807309 Correct: 94.30% Correct test: 67.0%
Epoch: 90 Loss: 20.278339874746962 Correct: 95.56% Correct test: 68.0%
Epoch: 100 Loss: 17.110141901426232 Correct: 96.20% Correct test: 66.0%
Epoch: 110 Loss: 14.489832302033335 Correct: 96.51% Correct test: 67.0%
Epoch: 120 Loss: 12.093794714433898 Correct: 98.10% Correct test: 67.0%
Epoch: 130 Loss: 10.08482857081431 Correct: 98.41% Correct test: 66.0%
Epoch: 140 

KeyboardInterrupt: 

In [89]:
error_test

0.58999999999999997

In [90]:
test_target.shape[0]

100

In [91]:
np.sum(list(output_test==test_target[:,1].data))/test_target.shape[0]

0.58999999999999997

In [148]:
test_target

Variable containing:
    0     1
    1     0
    1     0
    1     0
    0     1
    1     0
    1     0
    1     0
    0     1
    0     1
    0     1
    1     0
    1     0
    0     1
    0     1
    1     0
    1     0
    1     0
    1     0
    1     0
    1     0
    1     0
    1     0
    0     1
    1     0
    0     1
    0     1
    0     1
    1     0
    0     1
    1     0
    1     0
    0     1
    0     1
    1     0
    1     0
    0     1
    1     0
    0     1
    0     1
    0     1
    0     1
    1     0
    1     0
    1     0
    0     1
    1     0
    1     0
    0     1
    0     1
    0     1
    0     1
    0     1
    1     0
    0     1
    0     1
    0     1
    0     1
    1     0
    0     1
    0     1
    0     1
    1     0
    0     1
    1     0
    1     0
    0     1
    1     0
    1     0
    0     1
    1     0
    0     1
    0     1
    1     0
    1     0
    1     0
    1     0
    1     0
    0     1
    0     1
    1     0
    0  

In [149]:
output_test


 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 100]

# Test

In [62]:
test_dataset,test_target=load_script.load_dataset(train=False)

In [63]:
test_target=test_target[:,1]>test_target[:,0]

In [64]:
test_dataset=test_dataset.view(100,1,28,50)

In [65]:
correct=0
for test_element,test_target_element in zip(test_dataset,test_target):
        
    test_element=test_element.view(1,1,28,-1)
    out=net(test_element)
    print(out)
    out=int(out[1]>out[0])
    if out==int(test_target_element):
        correct+=1
    
        

Variable containing:
 0.4565
 0.4637
[torch.FloatTensor of size 2]

Variable containing:
 0.9957
 0.0063
[torch.FloatTensor of size 2]

Variable containing:
 0.9543
 0.0398
[torch.FloatTensor of size 2]

Variable containing:
 0.1809
 0.6106
[torch.FloatTensor of size 2]

Variable containing:
 0.7388
 0.3197
[torch.FloatTensor of size 2]

Variable containing:
 0.9646
 0.0391
[torch.FloatTensor of size 2]

Variable containing:
 0.4145
 0.4550
[torch.FloatTensor of size 2]

Variable containing:
 0.9785
 0.0375
[torch.FloatTensor of size 2]

Variable containing:
 0.8524
 0.1456
[torch.FloatTensor of size 2]

Variable containing:
 0.9275
 0.0814
[torch.FloatTensor of size 2]

Variable containing:
 0.8628
 0.0186
[torch.FloatTensor of size 2]

Variable containing:
 0.6237
 0.5560
[torch.FloatTensor of size 2]

Variable containing:
 0.8408
 0.0258
[torch.FloatTensor of size 2]

Variable containing:
 0.0559
 0.6401
[torch.FloatTensor of size 2]

Variable containing:
 0.3375
 0.5821
[torch.Floa

In [66]:
correct

68